<a href="https://colab.research.google.com/github/Hudiii/compression-picture/blob/main/picture-compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kompresi Citra Digital Menggunakan SVD

Implementasi metode Singular Value Decomposition untuk kompresi gambar.

**Kelompok 14 - Informatika UNS**

## 1. Import Library

In [ ]:
!pip install gradio -q

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import gradio as gr

## 2. Fungsi Kompresi SVD

In [ ]:
def compress_channel(channel, k):
    """Kompresi satu channel dengan SVD"""
    U, sigma, Vt = np.linalg.svd(channel, full_matrices=False)

    U_k = U[:, :k]
    sigma_k = np.diag(sigma[:k])
    Vt_k = Vt[:k, :]

    compressed = U_k @ sigma_k @ Vt_k
    compressed = np.clip(compressed, 0, 255)

    return compressed.astype(np.uint8), sigma

def compress_image(image, k):
    """Kompresi gambar RGB atau grayscale"""
    if len(image.shape) == 2:
        compressed, sigma = compress_channel(image, k)
        return compressed, sigma
    else:
        channels = []
        for i in range(3):
            comp, sigma = compress_channel(image[:, :, i], k)
            channels.append(comp)
        return np.stack(channels, axis=2), sigma

## 3. Fungsi Evaluasi

In [ ]:
def calculate_mse(original, compressed):
    """Hitung Mean Square Error"""
    return np.mean((original.astype(float) - compressed.astype(float)) ** 2)

def calculate_psnr(mse):
    """Hitung Peak Signal-to-Noise Ratio"""
    if mse == 0:
        return float('inf')
    return 20 * np.log10(255.0 / np.sqrt(mse))

def get_original_size(image):
    """Hitung ukuran file asli dalam KB"""
    img = Image.fromarray(image)
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='PNG')
    return len(img_byte_arr.getvalue()) / 1024

def calculate_storage(image, k):
    """Hitung ukuran storage SVD"""
    if len(image.shape) == 2:
        m, n = image.shape
        channels = 1
    else:
        m, n, channels = image.shape

    storage_per_channel = (m * k) + k + (k * n)
    total_bytes = storage_per_channel * channels * 8
    return total_bytes / 1024

## 4. Fungsi Visualisasi

In [ ]:
def plot_results(sigma, k):
    """Plot singular values dan energi kumulatif"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3.5))

    ax1.plot(sigma[:200])
    ax1.axvline(x=k, color='r', linestyle='--', label=f'k={k}')
    ax1.set_xlabel('Index')
    ax1.set_ylabel('Singular Value')
    ax1.set_title('Singular Values')
    ax1.legend()
    ax1.grid(alpha=0.3)

    cumsum = np.cumsum(sigma**2) / np.sum(sigma**2)
    ax2.plot(cumsum)
    ax2.axvline(x=k, color='r', linestyle='--')
    ax2.axhline(y=cumsum[k-1], color='g', linestyle='--', alpha=0.5)
    ax2.set_xlabel('k')
    ax2.set_ylabel('Energi Kumulatif')
    ax2.set_title('Energi Kumulatif')
    ax2.grid(alpha=0.3)

    plt.tight_layout()
    return fig

## 5. Interface Aplikasi

In [ ]:
def process_image(image, k_value):
    if image is None:
        return None, None, "Upload gambar terlebih dahulu"

    original = np.array(image)
    k = int(k_value)

    # Kompresi
    compressed, sigma = compress_image(original, k)

    # Evaluasi
    mse = calculate_mse(original, compressed)
    psnr = calculate_psnr(mse)

    # Ukuran file
    original_size = get_original_size(original)
    compressed_size = calculate_storage(original, k)
    compression_ratio = (1 - compressed_size/original_size) * 100

    # Plot
    plot = plot_results(sigma, k)

    # Format hasil
    hasil = f"""
Hasil Kompresi (k = {k}):

Ukuran Sebelum: {original_size:.2f} KB
Ukuran Sesudah: {compressed_size:.2f} KB
Rasio Kompresi: {compression_ratio:.2f}%

MSE: {mse:.2f}
PSNR: {psnr:.2f} dB
    """

    return compressed, plot, hasil

# Buat interface
with gr.Blocks() as demo:
    gr.Markdown("# Kompresi Citra dengan SVD")

    with gr.Row():
        with gr.Column():
            input_img = gr.Image(label="Input Gambar", type="numpy")
            k_slider = gr.Slider(1, 300, value=50, step=1, label="Nilai k")
            btn = gr.Button("Proses")

        with gr.Column():
            output_img = gr.Image(label="Hasil Kompresi")

    with gr.Row():
        hasil_text = gr.Textbox(label="Metrik Evaluasi", lines=8)

    with gr.Row():
        plot_output = gr.Plot(label="Analisis Singular Values")

    btn.click(process_image, inputs=[input_img, k_slider],
              outputs=[output_img, plot_output, hasil_text])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://31db868f2d43909462.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
